In [18]:
import os
from os.path import join
import numpy as np
import pandas as pd

terrier_path =      r"C:\Users\anton\Terrier"
logs_folder =       r"C:\Users\anton\Documents\GitHub\information-retrieval\ae2\logs"
results_folder =    r"C:\Users\anton\Documents\GitHub\information-retrieval\ae2\results"
features_file =     r"C:\Users\anton\Documents\GitHub\information-retrieval\ae2\features.txt"
terrier_index =     r"C:\terrier_data\indices\blocks_fields_stemming"
training_topics =   r"C:\terrier_data\topics\training\topics"
validation_topics = r"C:\terrier_data\topics\validation\topics"
training_qrels =    r"C:\terrier_data\topics\training\qrels"
validation_qrels =  r"C:\terrier_data\topics\validation\qrels"
qrels =             r"C:\terrier_data\topics\HP04\qrels"

sh_or_bat = "sh"
if os.name == 'nt':
    sh_or_bat = "bat"

terrier = join(terrier_path, "bin", "trec_terrier.%s" % sh_or_bat)
teval = join(terrier_path, "bin", "trec_eval.%s" % sh_or_bat)
letor_file= join(results_folder, "tr.letor")

In [19]:
def write_log(file, content, command=None):
    with open(join(logs_folder, file), "w") as w:
        if command is not None:
            w.write(command + "\n\n")
        for line in content:
            w.write(line + "\n")

Next, we want to retrieve results for the training topics. In this, we are going to be calculating results with multiple features, as listed in the etc/features.list file, so we use a series of Matching classes: FatFull to make a FatResultSet (i.e. a ResultSet with extra posting information), and FatFeaturedScoringMatching to add the additional features, and return a FeaturedResultSet. We then add the document label from the qrels using LabelDecorator, and write the results in a LETOR-compatible results file using Normalised2LETOROutputFormat:

In [20]:
results_file = join(results_folder,"pl2_ltr.res")
command="%s -r -Dtrec.topics=%s " % (terrier, training_topics) + \
    "-Dtrec.results.file=%s -Dtrec.model=PL2 -Dterrier.index.path=%s " % (results_file,  terrier_index) + \
    "-Dtrec.matching=FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull " + \
    "-Dtrec.querying.outputformat=Normalised2LETOROutputFormat " + \
    "-Dquerying.postprocesses.order=QueryExpansion,org.terrier.learning.LabelDecorator " + \
    "-Dquerying.postprocesses.controls=labels:org.terrier.learning.LabelDecorator,qe:QueryExpansion " + \
    "-Dquerying.default.controls=labels:on " + \
    "-Dlearning.labels.file=%s " % training_qrels + \
    "-Dtrec.results.file=%s -Dproximity.dependency.type=SD " % letor_file + \
    "-Dfat.featured.scoring.matching.features=FILE -Dfat.featured.scoring.matching.features.file=%s " % features_file 
    
run_results = !$command
write_log("pl2_ltr_querying.log", run_results, command=command)

eval_file = join(results_folder, "pl2_ltr.eval")
command = "%s %s %s -q > %s" % (teval, training_qrels, results_file, eval_file)    

run_results = !$command
write_log("pl2_ltr_eval.log", run_results, command=command)

Lets a have a look at what was output into tr.letor: 
(maybe not, it is too big)

The header reports the name of the features. "score"”" means the model used to generate the sample, i.e. the first pass retrieval, in our case DPH. After the header, for each retrieved document for each query, there is a single line in the output. The label obtained from the qrels file is the first entry on each row.

We repeat the retrieval step for the validation queries, --this time from the 2003 TREC task--:

[Here](http://terrier.org/docs/v4.2/learning.html)

In [16]:
results_file = join(results_folder,"pl2.res")
command="%s -r -Dtrec.topics=%s " % (terrier, training_topics) + \
    "-Dtrec.results.file=%s -Dtrec.model=PL2 -Dterrier.index.path=%s " % (results_file,  terrier_index)
    
run_results = !$command
write_log("pl2_querying.log", run_results, command=command)

eval_file = join(results_folder, "pl2.eval")
command = "%s %s %s -q > %s" % (teval, training_qrels, results_file, eval_file)    

run_results = !$command
write_log("pl2_eval.log", run_results, command=command)